<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Iniciais 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [28]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"

In [70]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)
trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)

In [71]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
0,2443
1,2443


In [72]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x))).mean())
average_len

62

In [73]:
palavras_unicas = list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) 
quantidade_palavras_unicas = len( palavras_unicas )
quantidade_palavras_unicas

9144

#TF-IDF (NÃO CONSEGUIMOS USAR NO MODELO CNN)

In [125]:
# word level tf-idf
# td-idf gera matriz com 9140 colunas (4 palavras estão sendo perdidas ao usar o regex \w{1,})
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features= quantidade_palavras_unicas)
X = tfidf_vect.fit_transform(trainDF['text'])

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, trainDF['label'], test_size=0.3, random_state=2)

In [126]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_x_text, valid_x_text, train_y_text, valid_y_text = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.3, random_state=2)

In [127]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x_text), maxlen=average_len)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x_text), maxlen=average_len)

In [128]:
train_seq_x.shape

(3420, 62)

# CNN model

In [129]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label, epochs=10) #adicionamos epoca
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [135]:
def create_cnn():
    # Add an Input Layer
    #input_layer = layers.Input((24,))
    input_layer = layers.Input((average_len,1,))

    input_layer = layers.SpatialDropout1D(0.3)(input_layer)

    # Add the word embedding Layer
    #embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    #embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(input_layer)
    #conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='sgd', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
    return model

In [136]:
classifier = create_cnn()
accuracyCNN = train_model(classifier,train_seq_x, train_y, valid_seq_x, is_neural_net=True)
#accuracyCNN = train_model(classifier,train_x, train_y, valid_x, is_neural_net=True)
print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

Epoch 1/10
107/107 [==============================] - 2s 9ms/step - loss: 0.4875 - acc: 0.5111 - mae: 0.4886
Epoch 2/10
107/107 [==============================] - 1s 8ms/step - loss: 0.4956 - acc: 0.5044 - mae: 0.4958
Epoch 3/10
107/107 [==============================] - 1s 6ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 4/10
107/107 [==============================] - 1s 5ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 5/10
107/107 [==============================] - 1s 5ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 6/10
107/107 [==============================] - 1s 6ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 7/10
107/107 [==============================] - 1s 5ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 8/10
107/107 [==============================] - 1s 5ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 9/10
107/107 [==============================] - 1s 5ms/step - loss: 0.4965 - acc: 0.5035 - mae: 0.4965
Epoch 10/10
107/107